# Slumber

> I'm tired too. Here are some things that might help.

In [ ]:
#| default_exp slumber

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pyedflib, edfio, torch, zarr, warnings, datetime as dt, torch.nn.functional as F, dask.array as da, numpy as np, pandas as pd, plotly.graph_objects as go, plotly.express as px
from torch.utils.data import Dataset
from scipy.signal import resample
import torchaudio.functional as F_audio
from typing import Union
from pathlib import Path
from plotly_resampler import FigureWidgetResampler
from mne.io import read_raw_edf

from pftsleep.data_preprocessing import calculate_samples_mp, calculate_stats_all, interpolate_nan_clip
from pftsleep.signal import butterworth
from scipy.ndimage import median_filter


## Constants

In [ ]:
#| export
ALL_FREQUENCY_FILTERS = {'ECG':[0.5,40], 
            'ECG (LL-RA)':[0.5,40],
            'EKG':[0.5,40],
            'ECG (L-R)':[0.5,40],
            'EOG(L)':[0.3,30],
            'EOG-L':[0.3,30],
            'E1':[0.3,30],
            'LOC':[0.3,30], 
            'E1-M2':[0.3,30], 
            'E1-AVG':[0.3,30],
            'EMG':[0.3,30],
            'cchin_l':[0.3,30],
            'chin':[0.3,30],
            'EMG (L-R)': [0.3,30], 
            'EMG (1-2)': [0.3,30],
            'EMG (1-3)': [0.3,30], 
            'Chin3':[0.3,30],
            'C4-M1':[0.3,30],
            'C4_M1':[0.3,30],
            'EEG':[0.3,30],
            "EEG(sec)":[0.3,30],
            "EEG1":[0.3,30],
            "EEG2":[0.3,30],
            'EEG3':[0.3,30],
            'C3-M2':[0.3,30],
            'C3_M2':[0.3,30],
            'C4-AVG':[0.3,30],
            'SaO2':[0.4, None],
            'SpO2':[0.4, None],
            'spo2':[0.4, None],
            'THOR RES':[0.5, None],
            'Thor':[0.5, None],
            'thorax':[0.5, None],
            'Thoracic':[0.5, None],
            'Chest':[0.5, None],
            'ABDO RES':[0.5, None],
            'abdomen':[0.5, None],
            'Abdo':[0.5, None],
            'Abdominal':[0.5, None],
            'ABD':[0.5, None]
            }

VOLTAGE_CHANNELS = ['ECG', 'ECG (LL-RA)', 'EKG', 'ECG (L-R)', 'EOG(L)', 'EOG-L', 'E1', 'LOC', 'E1-M2', 'E1-AVG','EMG', 'cchin_l', 'chin', 'EMG (L-R)', 'EMG (1-2)', 'EMG (1-3)', 'Chin3', 'C4-M1', 'C4_M1', 'EEG', 'EEG1', 'EEG2', 'EEG3', 'C3-M2', 'C3_M2', 'C4-AVG']

# physiological possible ranges to clip/interpolate
CLIP_INTERPOLATE_RANGES = {'SaO2': {"phys_range":[50,100], "percentiles":None}, 'SpO2':{"phys_range":[50,100], "percentiles":None}}

# wsc channels for training, note it does not have C4_M1
WSC_CHANNELS = [['ECG'], ['E1'], ['cchin_l', 'chin'], ['C3_M2'], ['spo2'], ['thorax'], ['abdomen']]

# apples channels for training, double checked and in the APPLES paper, LOC is referenced with M2
## see: https://academic.oup.com/sleep/article/34/3/303/2433804 and https://gitlab-scm.partners.org/zzz-public/nsrr/-/tree/master/studies/apples
APPLES_CHANNELS = [["ECG"], ["LOC"], ["EMG"], ["C4_M1", "C3_M2"], ["SpO2"], ["thorax"], ["abdomen"]]

# mesa channels for training, note that it does not have C3_M2, EEG3 == C4_M1
## note also that mesa refereences E1-fpz! not E!-M@
MESA_CHANNELS = [["EKG"], ["EOG-L"], ["EMG"], ["EEG3"], ["SpO2"], ["Thor"], ["Abdo"]]

# shhs channels for training 
## note that EOG(L) is referenced with PG1 (ground)
## EEG is C4_M1 and EEG(sec) is C3_M2
SHHS_CHANNELS = [["ECG"], ["EOG(L)"], ["EMG"], ["EEG", "EEG(sec)"], ["SaO2"], ["THOR RES"], ["ABDO RES"]]

# most of these weere derived in the zarr job file
MROS_CHANNELS = [['ECG (L-R)'], ['E1-M2'], ['EMG (L-R)'], ["C4-M1", "C3-M2"], ['SaO2', 'SpO2'], ['Thoracic', 'Chest'], ['Abdominal', 'ABD']]


## EDF Helpers

### Read EDFs

In [ ]:
#| export
def read_edf(file_path, # file path of edf
             channels=None, # channels in edf to read, will raise warning if channels do not exist
             frequency=None # frequency to resample all signals to
             ): # tuple of signals and header dictionary
    """
    Function to read an edf file and return a list of signals and header with the option to resample to a passed frequency
    """
    pyedflib.close_file(0) # ensure files are closed
    with pyedflib.EdfReader(file_path) as f:
        num_channels = f.signals_in_file
        if channels is not None:
            available_channels = [channel.upper() for channel in f.getSignalLabels()]
            channels = [channel.upper() for channel in channels]
            if len(set(channels) - set(available_channels)) != 0:
                warnings.warn(f'Missing channels {set(channels) - set(available_channels)}')
            channels_to_get = list(set(channels) & set(available_channels))
            channels_idxs = [available_channels.index(c) for c in channels_to_get]
        else:
            channels_idxs = range(num_channels)
        header = f.getHeader()
        header['Duration'] = f.getFileDuration()
        header['SignalHeaders'] = [f.getSignalHeaders()[c] for c in channels_idxs]
        signals = []
        if frequency is not None:
            required_length = int(header['Duration'] * frequency)
        for idx, c in enumerate(channels_idxs):
            signal = f.readSignal(c, digital=False)
            if frequency is not None and len(signal) != required_length:
                signal = resample(signal, required_length)
                header['SignalHeaders'][idx]['sample_rate'] = frequency
                header['SignalHeaders'][idx]['sample_frequency'] = frequency
            signals.append(signal)
    return signals, header

    #full_range = pd.date_range(header['startdate'], periods=header['Duration'], freq='1S')

In [ ]:
#| export
def read_edf_mne(file_path, # file path of edf
             channels=None, # channels in edf to read, will raise warning if channels do not exist
             frequency=None # frequency to resample all signals to
             )->Union[list, dict]: # tuple of signals and header dictionary
    """
    function to read edf with mne library
    i dont recommend using this. Use edfio instead.
    """
    raw_edf = read_raw_edf(file_path)
    signal_labels = raw_edf.ch_names
    num_channels = len(signal_labels)
    if channels is not None:
        available_channels = [channel.upper() for channel in signal_labels]
        channels = [channel.upper() for channel in channels]
        if len(set(channels) - set(available_channels)) != 0:
            warnings.warn(f'Missing channels {set(channels) - set(available_channels)}')
        channels_to_get = list(set(channels) & set(available_channels))
        channels_idxs = [available_channels.index(c) for c in channels_to_get]
    else:
        channels_idxs = range(num_channels)
    info_dict = dict(raw_edf.info)
    header = {'technician': info_dict.get('experimenter'),
        'recording_additional': '',
        'patientname': info_dict.get('subject_info').get('last_name'),
        'patient_additional': '',
        'patientcode': '',
        'equipment': '',
        'admincode': '',
        'sex': 'Female' if info_dict.get('subject_info').get('sex') == 0 else 'Male',
        'startdate': info_dict.get('meas_date'),
        'birthdate': info_dict.get('subject_info').get('birthday'),
        'gender': 'Female' if info_dict.get('subject_info').get('sex') == 0 else 'Male',
        'Duration':int(len(raw_edf) / info_dict.get('sfreq')),
        'SignalHeaders':[{'label':i['ch_name'], 'sample_frequency':info_dict.get('sfreq'), 'sample_rate':info_dict.get('sfreq')} for i in info_dict.get('chs')]
    }
    signals = []
    if frequency is not None:
        required_length = header['Duration'] * frequency
    for c in channels_idxs:
        signal = raw_edf[c][0][0]
        if frequency is not None and len(signal) != required_length:
            signal = resample(signal, required_length)
        signals.append(signal)
    return signals, header

In [ ]:
#| export
def read_edf_edfio(file_path, # file path of edf
             channels=None, # channels in edf to read, will raise warning if channels do not exist
             frequency=None # frequency to resample all signals to
             )->Union[list, dict]: # tuple of signals and header dictionary
    """
    function to read edfs with edfio
    """
    f = edfio.read_edf(file_path, lazy_load_data=True)
    num_channels = f.num_signals
    if channels is not None:
        available_channels = [channel.upper() for channel in f.labels]
        channels = [channel.upper() for channel in channels]
        if len(set(channels) - set(available_channels)) != 0:
            warnings.warn(f'Missing channels {set(channels) - set(available_channels)}')
        channels_to_get = list(set(channels) & set(available_channels))
        channels_idxs = [available_channels.index(c) for c in channels_to_get]
    else:
        channels_idxs = range(num_channels)
    header = {'technician': '' if f.recording.get_subfield(2) == 'X' else f.recording.get_subfield(2),
            'recording_additional': '',
            'patientname': '' if f.patient.get_subfield(3) == 'X' else f.patient.get_subfield(3),
            'patient_additional': '',
            'patientcode': '',
            'equipment': '',
            'admincode': '',
            'sex': '' if f.patient.get_subfield(1) == 'X' else f.patient.get_subfield(1).replace('M', 'Male').replace('F', 'Female'),
            'startdate': dt.datetime.combine(f.startdate, f.starttime),
            'birthdate': f.patient.get_subfield(2).lower().replace('-',' ') if f.patient.get_subfield(2) != 'X' else '',
            'gender': '' if f.patient.get_subfield(1) == 'X' else f.patient.get_subfield(1).replace('M', 'Male').replace('F', 'Female'),
            'Duration': int(f.duration),
            'SignalHeaders':[{'label':f.signals[i].label, 
                            'dimension':f.signals[i].physical_dimension,
                            'sample_rate':f.signals[i].sampling_frequency,
                            'sample_frequency':f.signals[i].sampling_frequency,
                            'physical_max': f.signals[i].physical_max,
                            'physical_min': f.signals[i].physical_min,
                            'digital_max': f.signals[i].digital_max,
                            'digital_min': f.signals[i].digital_min,
                            'prefilter': f.signals[i].prefiltering,
                            'transducer': f.signals[i].transducer_type
                            } for i in channels_idxs]
    }
    signals = []
    if frequency is not None:
        required_length = header['Duration'] * frequency
    for c in channels_idxs:
        signal = f.signals[c].data
        if frequency is not None and len(signal) != required_length:
            signal = resample(signal, required_length)
            header['SignalHeaders'][c]['sample_rate'] = frequency
            header['SignalHeaders'][c]['sample_frequency'] = frequency
        signals.append(signal)
    return signals, header
    

### Read Hypnograms

In [ ]:
#| export
def read_hypnogram(file, # file path of the hypnogram csv
                   epoch_length = None # epoch length of the hypnogram measurements, if passed will repeat this many times at each element
                   )->np.array: # numpy array of hypnogram
     """
     Function that reads a hypnogram csv and returns a numpy array of the hypnogram with optional repeats
     """
     y = pd.read_csv(file, header=None, names=[1])[1].to_numpy()
     if epoch_length is not None:
         y = y.repeat(epoch_length)
     return y

### EDFs to Zarr

In [ ]:
#| export
def edf_signals_to_zarr(edf_file_path, write_data_dir, overwrite=False, channels=None, channel_name_map=None, frequency=None, hyp_epoch_length=30, hyp_data_dir=None):
    """
    Function that converts an edf to a zarr file

    try_mne: tries to load files with mne instead of pyedflib (if there is an error). This seems dangerous as mne converts units (and potentially resamples, while pyedflib does not)
    """
    if channel_name_map is None:
        channel_name_map = {}
    edf_file_path = Path(edf_file_path)
    try:
        signals, header = read_edf(str(edf_file_path), channels=channels, frequency=frequency)
    except Exception as e:
        # try with edfio library, mne library is meh, because mne resamples data to highest frequency
        # then it adjusts units. edfio and pyedflib do not do this.
        signals, header = read_edf_edfio(str(edf_file_path), channels=channels, frequency=frequency)
    store = zarr.DirectoryStore(str(Path(write_data_dir)/edf_file_path.stem) + '.zarr')
    if overwrite:
        root_grp = zarr.group(store, overwrite=True)
    else:
        # this should not replace data, so we can continue to append to this group.
        root_grp = zarr.group(store, overwrite=False)
    signal_headers = header.pop('SignalHeaders')
    root_grp.attrs['header'] = {k:str(v) for k,v in header.items()}
    # get y/sleep staging data
    try:
        # try to get hypnogram data if it exists
        if hyp_data_dir is not None:
            hyp_data_dir = Path(hyp_data_dir)
            hyp_file_path = hyp_data_dir/Path(edf_file_path.stem + "-hyp.csv")
        else:
            hyp_file_path = edf_file_path.parent/Path(edf_file_path.stem + "-hyp.csv") # [V] V is variable per file and does not necessarily == patch_num
        
        if frequency is not None:
            y = read_hypnogram(hyp_file_path, epoch_length=(hyp_epoch_length*frequency))
        else:
            y = read_hypnogram(hyp_file_path, epoch_length=(hyp_epoch_length))

        if frequency is not None and len(y) < len(signals[-1]):
            # hypnogram is shorter, trim signals
            signals = signals[:, :len(y)] # [n_vars x T]
        elif frequency is not None and len(y) > len(signals[-1]):
            # signals are longer, trim hypnogram
            y = y[:len(signals[-1])] # [T]
    except:
        y = None
        pass
    # write signals
    for i, h in zip(signals, signal_headers):
        a = da.from_array(np.array(i, dtype=np.float16), chunks='auto') # read using dask
        name = h['label'] if h['label'] not in channel_name_map else channel_name_map[h['label']]
        h['mapped_label'] = name
        a.to_zarr(url=store, component=name, compute=True) # convert to zarr format
        root_grp[name].attrs['signal_header'] = h # assign metadata

    # write hypnogram
    if y is not None:
        a = da.from_array(y, chunks='auto')
        a.to_zarr(url=store, component='hypnogram', compute=True)
    zarr.consolidate_metadata(store)
    return root_grp

## Datasets

### Self Supervised Dataset

In [ ]:
#| export
def trim_wake_epochs_from_hypnogram(hypnogram, padding_mask=-100):
    """
    Function to trim wake epochs (if wake is the largest class) from hypnograms
    This function trims the wake epochs from the beginning and/or end of the hypnogram

    Adapted from Phan et al L-SeqSleepNet
    """
    # Check if Wake (stage 0) is the largest class
    idx, counts = hypnogram.unique(return_counts=True)
    counts = counts.numpy()
    if 0 not in idx:
        return hypnogram
    wake_idx = torch.where(idx==0)[0][0].item()
    n_wakes = counts[wake_idx].item()
    hypnogram = hypnogram.numpy()
    if n_wakes > np.max(counts[wake_idx+1:]):
        second_largest = np.max(counts[wake_idx+1:])
        
        # Create boolean array for Wake indices (True where stages == 0)
        W_ind = (hypnogram == 0)
        
        # Find first transition from/to Wake
        transitions = np.diff(W_ind.astype(int))
        last_evening_W_index = np.where(transitions != 0)[0][0]
        
        # Calculate number of evening Wake epochs
        num_evening_W = last_evening_W_index + 1 if hypnogram[0] == 0 else 0
        
        # Find last transition from/to Wake
        first_morning_W_index = np.where(transitions != 0)[0][-1] + 1
        num_morning_W = len(hypnogram) - first_morning_W_index
        
        nb_pre_post_sleep_wake_eps = num_evening_W + num_morning_W
        
        if nb_pre_post_sleep_wake_eps > second_largest:
            total_W_to_remove = nb_pre_post_sleep_wake_eps - second_largest
            
            if num_evening_W >= total_W_to_remove:
                # Remove from beginning only
                hypnogram[:total_W_to_remove] = padding_mask
            else:
                # Remove from both ends
                evening_W_to_remove = num_evening_W
                morning_W_to_remove = total_W_to_remove - evening_W_to_remove
                hypnogram[:evening_W_to_remove] = padding_mask
                hypnogram[-morning_W_to_remove:] = padding_mask

    return torch.from_numpy(hypnogram)


def trim_wake_epochs_from_signals(X, hypnogram, sequence_padding_mask, resampled_hypnogram_length, mask_x_with_zeros=False, padding_mask=-100):
    """
    Function to trim wake epochs (if wake is the largest class) from signals

    X: bs, channels, seq_len
    hypnogram: bs, seq_len / resampled_hypnogram_length
    sequence_padding_mask: bs, seq_len
    """
    # find invalid exact timepoints
    invalid_segments = hypnogram.repeat_interleave(resampled_hypnogram_length) == padding_mask
    if mask_x_with_zeros:
        X[:, invalid_segments] = 0
    sequence_padding_mask[:, invalid_segments] = 1
    return X, sequence_padding_mask


In [ ]:
#| export
class SelfSupervisedTimeFrequencyDataset(Dataset):
    def __init__(self, 
                 zarr_files, # zarr files that include samples
                 channels, # channels to use
                 max_seq_len_sec, # maximum sequence length (in seconds) to use (this is especially relevant when you are returning both stft and raw ts data to keep them in sync)
                 sample_seq_len_sec, # if no sample_df, generate sequences of this length in seconds as one sample
                 sample_stride_sec, #  if no sample_df, seconds of overlap for samples from the same array, if seq_len_seconds == overlap_seconds, there is no overlap
                 start_offset_sec=0, # number of seconds to exclude from beginning of sleep studies
                 trim_wake_epochs=True, # indicator to trim wake epochs from hypnograms, if it is the largest class
                 include_partial_samples=True, # indicator to include data from partial samples when return_full_length is false
                 sample_df=None, # dataframe indicating which indices within each zarr file includes a sample
                 frequency=125, # frequency of underlying data
                 return_hypnogram_every_sec=30, # integer value indicating the step in indexing in seconds
                 hypnogram_padding_mask=-100, # padded value to add to target and indice to ignore when computing loss
                 hypnogram_frequency=125, # frequency of underlying y hypnogram data
                 butterworth_filters=None, # dictionary of low pass, high pass, and bandpass dictionary to perform on channels
                 median_filter_kernel_size=None, # if not none, will apply median filter with kernel size
                 voltage_channels=VOLTAGE_CHANNELS, # if not None, these channels units will be looked at and changed to microvolts from mv uv etc.
                 clip_interpolations=None, # dictionary of channels:{'phys_range':..., 'percentiles':...} for filtering and interpolation of filtered values
                 scale_channels=False, # indicator to scale channels to the mean and std of the zarr files. 
                 time_channel_scales=None, # dictionary of channel:mean and channel:std values for scaling. Should use training statistics 
                 return_sequence_padding_mask=False, # indicator to return the key padding mask for attention masking
                 ):
        self.max_seq_len = max_seq_len_sec*frequency
        self.max_seq_len_sec = max_seq_len_sec
        self.include_partial_samples = include_partial_samples
        self.zarr_files = zarr_files
        self.channels = channels
        self.channels_has_dim = any(isinstance(i, list) for i in self.channels)
        self.sample_seq_len_sec = sample_seq_len_sec
        self.sample_seq_len = sample_seq_len_sec * frequency
        self.frequency = frequency
        self.sample_stride_sec = sample_stride_sec
        self.scale_channels = scale_channels
        self.clip_interpolations = clip_interpolations
        self.return_hypnogram_every_sec = return_hypnogram_every_sec
        self.hypnogram_padding_mask = hypnogram_padding_mask
        self.hypnogram_frequency = hypnogram_frequency
        self.return_sequence_padding_mask = return_sequence_padding_mask
        self.start_offset_sec = start_offset_sec
        self.trim_wake_epochs = trim_wake_epochs

        self.butterworth_filters = butterworth_filters
        self.median_filter_kernel_size = median_filter_kernel_size
        self.voltage_channels = voltage_channels
        
        if sample_df is None:
            print(f"Calculating samples with {sample_seq_len_sec} sec length and {sample_stride_sec} sec stride with {max_seq_len_sec} sec max")
            self.sample_df, self.total_samples = calculate_samples_mp(zarr_files, channels=channels, max_seq_len_sec=max_seq_len_sec, sample_seq_len_sec=sample_seq_len_sec, frequency=frequency, start_offset_sec=start_offset_sec, stride_sec=sample_stride_sec, include_partial_samples=include_partial_samples)
        else:
            assert 'file' in sample_df, "The `sample_df` must have a column `file` with the zarr file path."
            missing_in_zarrs = len(set(sample_df['file']) - set(zarr_files))
            missing_in_df = len(set(zarr_files) - set(sample_df['file']))
            if missing_in_zarrs > 0 or missing_in_df > 0:
                warnings.warn(f"There are {missing_in_zarrs} `zarr_files` not in the `sample_df` and {missing_in_df} files in the `sample_df` that arent in the `zarr_files`, they will be ignored.")
            sample_df = sample_df.loc[sample_df['file'].isin(zarr_files)].copy().reset_index(drop=True)
            self.sample_df = sample_df
            self.total_samples = len(sample_df)
        
        if self.scale_channels:
            if time_channel_scales is None or time_channel_scales == {}:
                print(f"Calculating sample statistics for {channels}")
                channel_scales_df = calculate_stats_all(zarr_files=self.sample_df.file.tolist(), channels=channels, sample_wise=False, channel_magnitude_multiple=None)
                self.time_channel_scales = channel_scales_df.to_dict(orient='index')
            else:
                self.time_channel_scales = time_channel_scales
            
    def __len__(self):
        return self.total_samples
    
    def __getitem__(self, idx):
        # get full length x, idx can be a slice
        sample = self.sample_df.iloc[idx]
        root_grp = zarr.open(sample['file'])
        duration_seconds = int(root_grp.attrs.asdict()['header']['Duration'])
        signals = []
        channels = []
        if self.channels_has_dim:
            avail_channels = list(root_grp.array_keys())
            for p in self.channels:
                channels.append(next(x for x in p if x in avail_channels))
        else:
            channels = self.channels
        for channel in channels:
            channel_frequency = root_grp[channel].attrs.asdict()['signal_header']['sample_frequency']
            channel_dimension = root_grp[channel].attrs.asdict()['signal_header']['dimension'] # whether in mV, microv or v
            if channel_frequency != self.frequency:
                start_idx, end_idx = int(sample['start_idx'] / self.frequency * channel_frequency), int(sample['end_idx'] / self.frequency * channel_frequency)
            else:
                start_idx, end_idx = sample['start_idx'], sample['end_idx']
            temp = np.array(root_grp[channel][start_idx:end_idx], dtype=np.float32)
            if channel_frequency < self.frequency:
                # need to upsample first, then apply filters
                #temp = resample(root_grp[channel][start_idx:end_idx], self.frequency*self.sample_seq_len_sec) if duration_seconds >= self.sample_seq_len_sec else resample(root_grp[channel][start_idx:end_idx], duration_seconds*self.frequency)
                temp = F_audio.resample(torch.from_numpy(temp), channel_frequency, self.frequency).numpy()
                if self.median_filter_kernel_size is not None:
                    temp = median_filter(temp, size=self.median_filter_kernel_size, mode='nearest')
                if self.butterworth_filters is not None and channel in self.butterworth_filters:
                    freq_range = self.butterworth_filters[channel]
                    btype = 'highpass' if freq_range[0] is None else 'lowpass' if freq_range[1] is None else 'bandpass'
                    freq_range = freq_range[1] if freq_range[0] is None else freq_range[0] if freq_range[1] is None else freq_range
                    temp = butterworth(temp, freq_range=freq_range, btype=btype, fs=self.frequency, order=2)
            elif channel_frequency > self.frequency:
                # need to upsample or downsample, filter first, then resample
                if self.median_filter_kernel_size is not None:
                    temp = median_filter(temp, size=self.median_filter_kernel_size, mode='nearest')
                if self.butterworth_filters is not None and channel in self.butterworth_filters:
                    freq_range = self.butterworth_filters[channel]
                    btype = 'highpass' if freq_range[0] is None else 'lowpass' if freq_range[1] is None else 'bandpass'
                    freq_range = freq_range[1] if freq_range[0] is None else freq_range[0] if freq_range[1] is None else freq_range
                    temp = butterworth(temp, freq_range=freq_range, btype=btype, fs=channel_frequency, order=2)
                temp = F_audio.resample(torch.from_numpy(temp), channel_frequency, self.frequency).numpy()
            else:
                if self.median_filter_kernel_size is not None:
                    temp = median_filter(temp, size=self.median_filter_kernel_size, mode='nearest')
                if self.butterworth_filters is not None and channel in self.butterworth_filters:
                    freq_range = self.butterworth_filters[channel]
                    btype = 'highpass' if freq_range[0] is None else 'lowpass' if freq_range[1] is None else 'bandpass'
                    freq_range = freq_range[1] if freq_range[0] is None else freq_range[0] if freq_range[1] is None else freq_range
                    temp = butterworth(temp, freq_range=freq_range, btype=btype, fs=self.frequency, order=2)
            if channel in self.voltage_channels:
                if channel_dimension.lower() == 'mv':
                    temp = temp * 1e3
            if self.clip_interpolations is not None and channel in self.clip_interpolations:
                temp = interpolate_nan_clip(temp, physiological_range_clip=self.clip_interpolations[channel]['phys_range'], percentile_clip=self.clip_interpolations[channel]['percentiles'])
            if not self.scale_channels:
                signals.append(temp)
            else:
                signals.append((temp - self.time_channel_scales[channel]['mean'])/self.time_channel_scales[channel]['std'])
        X = Y = torch.from_numpy(np.stack(signals, dtype=np.float32))
        if self.trim_wake_epochs and 'hypnogram' in root_grp.array_keys():
            if self.hypnogram_frequency == self.frequency:
                # hypnogram has already been resampled
                hypnogram = torch.from_numpy(np.array(root_grp['hypnogram'][sample['start_idx']:sample['end_idx']], dtype=np.int64))
            else:
                # hypnogram has sampling rate of something else
                hyp_start_idx = int((sample['start_idx'] / self.frequency) * self.hypnogram_frequency)
                hyp_end_idx = int((sample['end_idx'] / self.frequency) * self.hypnogram_frequency)
                hypnogram = np.array(root_grp['hypnogram'][hyp_start_idx:hyp_end_idx], dtype=np.int64)
                hypnogram = torch.from_numpy(hypnogram)
        
            expected_hyp_len = int((self.sample_seq_len / self.frequency) * self.hypnogram_frequency)
            if hypnogram.shape[-1] < expected_hyp_len:
                hypnogram = F.pad(hypnogram, (0, expected_hyp_len - hypnogram.shape[-1]), 'constant', value=self.hypnogram_padding_mask)
            
            samples_per_epoch = int(self.return_hypnogram_every_sec * self.hypnogram_frequency)
            hypnogram = hypnogram[::samples_per_epoch]
            hypnogram[hypnogram>5] = self.hypnogram_padding_mask # replace values > 5 with -100 (there are 8 (6 and 9 are unknown and wake, stage 4 is also present, REM is 5) classes total in shhs and they encode movement/wake as 9 and 0 as wake, this combines the two)
        
        sequence_padding_mask = torch.zeros([1,X.shape[-1]])
        if X.shape[-1] < self.sample_seq_len:
            sequence_padding_mask = F.pad(sequence_padding_mask, (0,self.sample_seq_len - sequence_padding_mask.shape[-1]), 'constant', value=1) # constant pad with 1
            X = Y = F.pad(X, (0,self.sample_seq_len - X.shape[-1]), 'constant', value=0) # replicate pad
        
        # check if wake is the largest class and assign padding mask to wake epochs if so
        if self.trim_wake_epochs and 'hypnogram' in root_grp.array_keys():
            resampled_hypnogram_length = int(self.return_hypnogram_every_sec * self.frequency)
            hypnogram = trim_wake_epochs_from_hypnogram(hypnogram, padding_mask=self.hypnogram_padding_mask)
            X, sequence_padding_mask = trim_wake_epochs_from_signals(X, hypnogram=hypnogram, sequence_padding_mask=sequence_padding_mask, resampled_hypnogram_length=resampled_hypnogram_length, mask_x_with_zeros=False, padding_mask=self.hypnogram_padding_mask)
            X = Y
        if torch.isnan(X).any():
            warnings.warn(f"X has nan values, sample_idx: {idx}")
        if self.return_sequence_padding_mask:
            return X, Y, sequence_padding_mask
        else:
            return X, Y

### Sleep Stage Supervised Dataset

In [ ]:
#| export
class HypnogramTimeFrequencyDataset(Dataset):
    def __init__(self, 
                 zarr_files, # zarr files that include samples
                 channels, # channels to use
                 max_seq_len_sec, # maximum sequence length (in seconds) to use (this is especially relevant when you are returning both stft and raw ts data to keep them in sync)
                 sample_seq_len_sec, # if no sample_df, generate sequences of this length in seconds as one sample
                 sample_stride_sec, #  if no sample_df, seconds of overlap for samples from the same array, if seq_len_seconds == overlap_seconds, there is no overlap
                 start_offset_sec=0, # number of seconds to exclude from beginning of sleep studies
                 trim_wake_epochs=True, # indicator to trim wake epochs from hypnograms, if it is the largest class
                 include_partial_samples=True, # indicator to include data from partial samples when return_full_length is false
                 sample_df=None, # dataframe indicating which indices within each zarr file includes a sample
                 frequency=125, # frequency of underlying data
                 return_y_every_sec=30, # integer value indicating the step in indexing in seconds
                 y_padding_mask=-100, # padded value to add to target and indice to ignore when computing loss
                 y_frequency=125, # frequency of underlying y hypnogram data
                 butterworth_filters=None, # dictionary of low pass, high pass, and bandpass dictionary to perform on channels
                 median_filter_kernel_size=None, # if not none, will apply median filter with kernel size
                 voltage_channels=VOLTAGE_CHANNELS, # if not None, these channels units will be looked at and changed to microvolts from mv uv etc.
                 clip_interpolations=None, # dictionary of channels:{'phys_range':..., 'percentiles':...} for filtering and interpolation of filtered values
                 scale_channels=False, # indicator to scale channels to the mean and std of the zarr files. 
                 time_channel_scales=None, # dictionary of channel:mean and channel:std values for scaling. Should use training statistics 
                 return_sequence_padding_mask=False, # indicator to return the key padding mask for attention masking
                 ):
        self.max_seq_len = max_seq_len_sec*frequency
        self.max_seq_len_sec = max_seq_len_sec
        self.include_partial_samples = include_partial_samples
        self.zarr_files = zarr_files
        self.channels = channels
        self.channels_has_dim = any(isinstance(i, list) for i in self.channels)
        self.sample_seq_len_sec = sample_seq_len_sec
        self.sample_seq_len = sample_seq_len_sec * frequency
        self.frequency = frequency
        self.sample_stride_sec = sample_stride_sec
        self.scale_channels = scale_channels
        self.clip_interpolations = clip_interpolations
        self.return_y_every_sec = return_y_every_sec
        self.y_padding_mask = y_padding_mask
        self.y_frequency = y_frequency
        self.return_sequence_padding_mask = return_sequence_padding_mask
        self.start_offset_sec = start_offset_sec
        self.trim_wake_epochs = trim_wake_epochs

        self.butterworth_filters = butterworth_filters
        self.median_filter_kernel_size = median_filter_kernel_size
        self.voltage_channels = voltage_channels
        
        if sample_df is None:
            print(f"Calculating samples with {sample_seq_len_sec} sec length and {sample_stride_sec} sec stride with {max_seq_len_sec} sec max")
            self.sample_df, self.total_samples = calculate_samples_mp(zarr_files, channels=channels, max_seq_len_sec=max_seq_len_sec, sample_seq_len_sec=sample_seq_len_sec, frequency=frequency, start_offset_sec=start_offset_sec, stride_sec=sample_stride_sec, include_partial_samples=include_partial_samples)
        else:
            assert 'file' in sample_df, "The `sample_df` must have a column `file` with the zarr file path."
            missing_in_zarrs = len(set(sample_df['file']) - set(zarr_files))
            missing_in_df = len(set(zarr_files) - set(sample_df['file']))
            if missing_in_zarrs > 0 or missing_in_df > 0:
                warnings.warn(f"There are {missing_in_zarrs} `zarr_files` not in the `sample_df` and {missing_in_df} files in the `sample_df` that arent in the `zarr_files`, they will be ignored.")
            sample_df = sample_df.loc[sample_df['file'].isin(zarr_files)].copy().reset_index(drop=True)
            self.sample_df = sample_df
            self.total_samples = len(sample_df)
        
        if self.scale_channels:
            if time_channel_scales is None or time_channel_scales == {}:
                print(f"Calculating sample statistics for {channels}")
                channel_scales_df = calculate_stats_all(zarr_files=self.sample_df.file.tolist(), channels=channels, sample_wise=False, channel_magnitude_multiple=channel_magnitude_multiple)
                self.time_channel_scales = channel_scales_df.to_dict(orient='index')
            else:
                self.time_channel_scales = time_channel_scales
            
    def __len__(self):
        return self.total_samples
    
    def __getitem__(self, idx):
        # get full length x, idx can be a slice
        sample = self.sample_df.iloc[idx]
        root_grp = zarr.open(sample['file'])
        duration_seconds = int(root_grp.attrs.asdict()['header']['Duration'])
        signals = []
        channels = []
        if self.channels_has_dim:
            avail_channels = list(root_grp.array_keys())
            for p in self.channels:
                channels.append(next(x for x in p if x in avail_channels))
        else:
            channels = self.channels
        for channel in channels:
            channel_frequency = root_grp[channel].attrs.asdict()['signal_header']['sample_frequency']
            channel_dimension = root_grp[channel].attrs.asdict()['signal_header']['dimension'] # whether in mV, microv or v
            if channel_frequency != self.frequency:
                start_idx, end_idx = int(sample['start_idx'] / self.frequency * channel_frequency), int(sample['end_idx'] / self.frequency * channel_frequency)
            else:
                start_idx, end_idx = sample['start_idx'], sample['end_idx']
            temp = np.array(root_grp[channel][start_idx:end_idx], dtype=np.float32)
            if channel_frequency < self.frequency:
                # need to upsample first, then apply filters
                #temp = resample(root_grp[channel][start_idx:end_idx], self.frequency*self.sample_seq_len_sec) if duration_seconds >= self.sample_seq_len_sec else resample(root_grp[channel][start_idx:end_idx], duration_seconds*self.frequency)
                temp = F_audio.resample(torch.from_numpy(temp), channel_frequency, self.frequency).numpy()
                if self.median_filter_kernel_size is not None:
                    temp = median_filter(temp, size=self.median_filter_kernel_size, mode='nearest')
                if self.butterworth_filters is not None and channel in self.butterworth_filters:
                    freq_range = self.butterworth_filters[channel]
                    btype = 'highpass' if freq_range[0] is None else 'lowpass' if freq_range[1] is None else 'bandpass'
                    freq_range = freq_range[1] if freq_range[0] is None else freq_range[0] if freq_range[1] is None else freq_range
                    temp = butterworth(temp, freq_range=freq_range, btype=btype, fs=self.frequency, order=2)
            elif channel_frequency > self.frequency:
                # need to upsample or downsample, filter first, then resample
                if self.median_filter_kernel_size is not None:
                    temp = median_filter(temp, size=self.median_filter_kernel_size, mode='nearest')
                if self.butterworth_filters is not None and channel in self.butterworth_filters:
                    freq_range = self.butterworth_filters[channel]
                    btype = 'highpass' if freq_range[0] is None else 'lowpass' if freq_range[1] is None else 'bandpass'
                    freq_range = freq_range[1] if freq_range[0] is None else freq_range[0] if freq_range[1] is None else freq_range
                    temp = butterworth(temp, freq_range=freq_range, btype=btype, fs=channel_frequency, order=2)
                temp = F_audio.resample(torch.from_numpy(temp), channel_frequency, self.frequency).numpy()
            else:
                if self.median_filter_kernel_size is not None:
                    temp = median_filter(temp, size=self.median_filter_kernel_size, mode='nearest')
                if self.butterworth_filters is not None and channel in self.butterworth_filters:
                    freq_range = self.butterworth_filters[channel]
                    btype = 'highpass' if freq_range[0] is None else 'lowpass' if freq_range[1] is None else 'bandpass'
                    freq_range = freq_range[1] if freq_range[0] is None else freq_range[0] if freq_range[1] is None else freq_range
                    temp = butterworth(temp, freq_range=freq_range, btype=btype, fs=self.frequency, order=2)
            if channel in self.voltage_channels:
                if channel_dimension.lower() == 'mv':
                    temp = temp * 1e3
            if self.clip_interpolations is not None and channel in self.clip_interpolations:
                temp = interpolate_nan_clip(temp, physiological_range_clip=self.clip_interpolations[channel]['phys_range'], percentile_clip=self.clip_interpolations[channel]['percentiles'])
            if not self.scale_channels:
                signals.append(temp)
            else:
                signals.append((temp - self.time_channel_scales[channel]['mean'])/self.time_channel_scales[channel]['std'])
        X = torch.from_numpy(np.stack(signals, dtype=np.float32))
        if self.y_frequency == self.frequency:
            # hypnogram has already been resampled
            Y = torch.from_numpy(np.array(root_grp['hypnogram'][sample['start_idx']:sample['end_idx']], dtype=np.int64))
        else:
            # hypnogram has sampling rate of something else
            hyp_start_idx = int((sample['start_idx'] / self.frequency) * self.y_frequency)
            hyp_end_idx = int((sample['end_idx'] / self.frequency) * self.y_frequency)
            Y = np.array(root_grp['hypnogram'][hyp_start_idx:hyp_end_idx], dtype=np.int64)
            Y = torch.from_numpy(Y)
        
        sequence_padding_mask = torch.zeros([1,X.shape[-1]])
        if X.shape[-1] < self.sample_seq_len:
            sequence_padding_mask = F.pad(sequence_padding_mask, (0,self.sample_seq_len - sequence_padding_mask.shape[-1]), 'constant', value=1) # constant pad with 1
            X = F.pad(X, (0,self.sample_seq_len - X.shape[-1]), 'constant', value=0) # replicate pad
        
        expected_hyp_len = int((self.sample_seq_len / self.frequency) * self.y_frequency)
        if Y.shape[-1] < expected_hyp_len:
            Y = F.pad(Y, (0, expected_hyp_len - Y.shape[-1]), 'constant', value=self.y_padding_mask)
        
        samples_per_epoch = int(self.return_y_every_sec * self.y_frequency)
        Y = Y[::samples_per_epoch]

        assert Y.max() <= 4 and (Y.min() == self.y_padding_mask or Y.min() >= 0), "There are values greater than 4 in the hypnogram or less than 0 that are not the padding mask. This is unexpected. 0-4 are the only valid values indicating wake, n1, n2, n3, rem, and a padding mask."
        # check if wake is the largest class and assign padding mask to wake epochs if so
        if self.trim_wake_epochs:
            resampled_hypnogram_length = int(self.return_y_every_sec * self.frequency)
            Y = trim_wake_epochs_from_hypnogram(Y, padding_mask=self.y_padding_mask)
            X, sequence_padding_mask = trim_wake_epochs_from_signals(X, hypnogram=Y, sequence_padding_mask=sequence_padding_mask, resampled_hypnogram_length=resampled_hypnogram_length, mask_x_with_zeros=False, padding_mask=self.y_padding_mask)
        if torch.isnan(X).any():
            warnings.warn(f"X has nan values, sample_idx: {idx}")
        if torch.isnan(Y).any():
            warnings.warn(f"Y has nan values, sample_idx: {idx}")
        if self.return_sequence_padding_mask:
            return X, Y, sequence_padding_mask
        else:
            return X, Y

## Plotting

In [ ]:
#| export
def plot_edf_signals(signals, signal_names, signal_comparisons=None, use_resampler=False, normalize=False, title_text='', colorscale=None):
    fig = go.Figure().set_subplots(len(signal_names), 1, shared_xaxes=True, vertical_spacing=0.01)
    if use_resampler:
        fig = FigureWidgetResampler(fig)
    n_colors = len(signals)
    if colorscale is not None:
        colors = px.colors.sample_colorscale(colorscale, [n/(n_colors-1) if n_colors != 1 else 0 for n in range(n_colors)])
    else:
        colors = ['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2']
    for idx, s in enumerate(signals):
        y = (s - s.min())/(s.max() - s.min()) if normalize else s
        fig.add_trace(go.Scattergl(name=signal_names[idx],  
                                    mode='markers+lines', 
                                    marker_opacity=0, line={'width': 1.5, 'color':colors[idx]},
                                    y=y),
                                    row=idx+1,
                                    col=1
                                    )
    if signal_comparisons is not None:
        for idx, s in enumerate(signal_comparisons):
            y = (s - s.min())/(s.max() - s.min()) if normalize else s
            fig.add_trace(go.Scattergl(name=None,#signal_names[idx],  
                                        mode='markers+lines', 
                                        showlegend=False,
                                        marker_opacity=0, line={'width': 1.5, 'color':'black'},
                                        y=y),
                                        row=idx+1,
                                        col=1
                                        )
    fig.update_layout(height=800, width=1500, 
                      title_text=f"{title_text}")
    return fig

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()